# Heart Disease Predictor


In [99]:
#importing all dependencies + logistic regression model
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint





In [100]:
#create a pandas dataframe for heart.csv

heartData = pd.read_csv("./heart.csv") 


Visualizing + Gaining Statistical Insights on the Dataset

In [69]:
heartData.head() #visualizing first 5 rows of the dataset


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [70]:
heartData.describe() #prints out five number summary for all catagories

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [71]:
heartData.shape #prints out the number of rows and columns (dataset has 1025 entries and 14 fields)


(303, 14)

In [101]:
#checking if there are any null values in our dataset 

heartData.info() #checking if there are any null values (there are no null/missing values in the dataset, so we can proceed)
heartData.isnull().sum() #alternative way of checking if there are other null values 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [102]:
#detecting any duplicates
duplicates = heartData.duplicated()
print(duplicates.sum())

duplicates = heartData[heartData.duplicated()]
print(duplicates)

1
     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
164   38    1   2       138   175    0        1      173      0      0.0   

     slope  ca  thal  target  
164      2   4     2       1  


In [103]:
#drop all duplicates
heartData = heartData.drop_duplicates()

duplicates = heartData.duplicated()
print(duplicates.sum())

0


In [104]:
#Getting a count of how many people have HD and how many people do not
heartData['target'].value_counts() 

target
1    164
0    138
Name: count, dtype: int64

Separate Dataset, One without Target and one with Target

In [105]:
dataSetWithoutTarget = heartData.drop(columns='target', axis=1)
target = heartData['target']

In [106]:
print("Data Without Target")
print(dataSetWithoutTarget)


Data Without Target
     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  
0        0   0     1  
1        0   0     2 

In [78]:

print("Target")
print(target)

Target
0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 302, dtype: int64


Separating Data into Training and Testing (80 percent for training, 20 percent for testing)



In [107]:
#splitting data into training and testing
dataSetWithoutTargetTrain, dataSetWithoutTargetTest, targetTrain, targetTest = train_test_split(dataSetWithoutTarget, target, random_state=42, test_size=0.2, stratify=target)

In [80]:
#visualizing sizes for all 
print("Original Dataset Size: " + str(heartData.shape))

print("Dataset Train Size: " + str(dataSetWithoutTargetTrain.shape))
print("Dataset Test Size: " + str(dataSetWithoutTargetTest.shape))
print("Target Train Size: " + str(targetTrain.shape))
print("Target Test Size: " + str(targetTest.shape))

Original Dataset Size: (302, 14)
Dataset Train Size: (241, 13)
Dataset Test Size: (61, 13)
Target Train Size: (241,)
Target Test Size: (61,)


# Training Models 

Training Logical Regression Model

In [ ]:
#takes about 1-2 minutes to run
lr = LogisticRegression()
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', None],
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga', 'lbfgs', 'newton-cg', 'sag'],
    'l1_ratio': [0.1, 0.5, 0.9],
    'max_iter': [100, 200, 500, 900],
    'class_weight': [None, 'balanced'],
    'fit_intercept': [True, False]
}

grid_search = GridSearchCV(lr, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid_search.fit(dataSetWithoutTargetTrain, targetTrain)
print("Best Parameters:", grid_search.best_params_)
print("Best F1 score (macro-averaged):", grid_search.best_score_)


Best Parameters: {'C': 0.1, 'class_weight': None, 'fit_intercept': True, 'l1_ratio': 0.1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Best F1 score (macro-averaged): 0.8293103510286693


In [138]:
LRmodel = LogisticRegression(C=  0.1, class_weight= None, fit_intercept= True, l1_ratio= 0.1,max_iter=100, penalty='l2', solver = 'liblinear')
LRmodel.fit(dataSetWithoutTargetTrain, targetTrain) #tries to find a pattern between all medical stats and whether or not the person has heart disease

LogisticRegression(C=0.1, l1_ratio=0.1, solver='liblinear')

### Evaluating LR Performance

In [139]:
#Training Data Accuracy

trainingDatasetPrediction = LRmodel.predict(dataSetWithoutTargetTrain)
trainingDataAccuracy = accuracy_score(trainingDatasetPrediction, targetTrain)

print("Training Data Accuracy: " + str(trainingDataAccuracy))


Training Data Accuracy: 0.8506224066390041


In [140]:
#Testing Data Accuracy 

testingDataPrediction = LRmodel.predict(dataSetWithoutTargetTest)
testingDataAccuracy = accuracy_score(testingDataPrediction, targetTest)

print("Testing Data Accuracy", testingDataAccuracy)
print(classification_report(targetTest, testingDataPrediction))



Testing Data Accuracy 0.7704918032786885
              precision    recall  f1-score   support

           0       0.79      0.68      0.73        28
           1       0.76      0.85      0.80        33

    accuracy                           0.77        61
   macro avg       0.77      0.76      0.77        61
weighted avg       0.77      0.77      0.77        61



Basic Testing System For LR Model (Will need to be changed to UI in the Future)

In [111]:
#comment out whichever model you want to test, use tests from csv file and see if it predicts


inputtedData = (56,0,1,140,294,0,0,153,0,1.3,1,0,2) #should mostly print out Person has heart disease
#inputtedData = (51,0,0,130,305,0,1,142,1,1.2,1,0,3) #should mostly print out Person does not have heart disease

reshapedNumPyArr = np.asarray(inputtedData).reshape(1,-1)

prediction = LRmodel.predict(reshapedNumPyArr)

print(prediction)

if(prediction[0] == 0):
    print("Person does not have heart disease")
else:
    print("Person has heart disease")



[1]
Person has heart disease


Training KNN Model 

In [112]:
#Preprocessing: Normalizing the features (KNN relies on distance)
scaler = StandardScaler()
X_train = scaler.fit_transform(dataSetWithoutTargetTrain)
X_test = scaler.transform(dataSetWithoutTargetTest)


In [130]:
#TAKES 13-14 minutes to run add n_iter = 100 to gridsearchcv to make it faster (only compares 100)
param_grid = {
    'n_neighbors': list(range(1, 31)),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski'], #we actually solved chebyshev in our class
    'p': [1, 2, 3],
    'leaf_size': [10, 20, 30, 50],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}


knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='f1_macro', return_train_score=True)
#cv = number of subsets data is divided into for cross validation
#scoring = whatever its optimizing, f1 or f1_macro
grid_search.fit(X_train, targetTrain)

print("Best parameters:", grid_search.best_params_)
print("Best F1 score (macro-averaged):", grid_search.best_score_)


Best parameters: {'algorithm': 'auto', 'leaf_size': 10, 'metric': 'manhattan', 'n_neighbors': 16, 'p': 1, 'weights': 'uniform'}
Best F1 score (macro-averaged): 0.8518347875230756


In [131]:
#fitting the model 
KNNModel = KNeighborsClassifier(algorithm='auto', leaf_size = 10, metric = 'manhattan',  n_neighbors=16, p =1, weights='uniform')
KNNModel.fit(X_train, targetTrain)

KNeighborsClassifier(leaf_size=10, metric='manhattan', n_neighbors=16, p=1)

### Evaluating Performance for KNN


In [132]:
#Training Data Accuracy 
knnTrainPred = KNNModel.predict(X_train)
knnTrainAcc = accuracy_score(knnTrainPred, targetTrain)

print("Training Data Accuracy", knnTrainAcc)


Training Data Accuracy 0.8423236514522822


In [133]:
#Testing Data Accuracy 
knnTestPred = KNNModel.predict(X_test)
knnTestAcc = accuracy_score(knnTestPred, targetTest)

print("Testing Data Accuracy", knnTestAcc)

print(classification_report(targetTest, knnTestPred))


Testing Data Accuracy 0.8032786885245902
              precision    recall  f1-score   support

           0       0.86      0.68      0.76        28
           1       0.77      0.91      0.83        33

    accuracy                           0.80        61
   macro avg       0.82      0.79      0.80        61
weighted avg       0.81      0.80      0.80        61



Simple Testing for KNN Model

In [89]:
#comment out whichever model you want to test

inputtedData = (57,1,0,140,192,0,1,148,0,0.4,1,0,1) #should mostly print out Person has heart disease
#inputtedData = (50,1,0,144,200,0,0,126,1,0.9,1,0,3) #should mostly print out Person does not have heart disease


reshapedNumPyArr = np.asarray(inputtedData).reshape(1,-1)

prediction = KNNModel.predict(reshapedNumPyArr)

print(prediction)

if(prediction[0] == 0):
    print("Person does not have heart disease")
else:
    print("Person has heart disease")



[1]
Person has heart disease


Training Random Forest Model 


In [148]:
#CANNOT USE GRIDSEARCH YOUR COMPUTER Will TaKE houRS
rf = RandomForestClassifier()

param_grid = {
    'n_estimators': [50, 100, 200, 500],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    'class_weight': [None, 'balanced', 'balanced_subsample']
}

random_search = RandomizedSearchCV(
    estimator=rf, 
    param_distributions=param_grid, 
    n_iter=100,  # Test 100 random combinations
    cv=5, 
    scoring='f1_macro', 
    n_jobs=-1, 
    random_state=42
)#njobs uses all processors cuz ur computer bouta blow
grid_search.fit(dataSetWithoutTargetTrain, targetTrain)

# Output the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 score (macro-averaged):", grid_search.best_score_)


KeyboardInterrupt: 

In [ ]:
#Preprocessing

#The randomized search CV and param_dist is commented out rn because uncommenting it gives us a training accuracy of 
# 1.0, which is DEFINITELY overfitting. The current parameters give us an accuracy of 96 which is still lowk high but we'll get more insight after you run your tests. 

param_dist = {'n_estimators': randint(50, 500), #n_estimators -> the number of trees to consider: uses a uniform distribution between 50 and 500.
              'max_depth': randint(1,20)}       #max_depth -> the max dept that the tree goes to make a decision, uses a uniform distribution between 1 and 20

rfModel = RandomForestClassifier()

#randomSearch = RandomizedSearchCV(rfModel, param_distributions=param_dist, n_iter=5, cv=5) #hyperparameter tuning (finding the optimal set of parameters for RF using the parameter distributions shown above)

rfModel.fit(dataSetWithoutTargetTrain, targetTrain)

RandomForestClassifier()

### Evaluating Performance for Random Forest


In [144]:
#Performance for Training Data
rfModelTrainPred = rfModel.predict(dataSetWithoutTargetTrain)
trainAccuracy = accuracy_score(rfModelTrainPred, targetTrain)

print("Training Data Accuracy: ", trainAccuracy)

Training Data Accuracy:  1.0


In [145]:
rfModelTestPred = rfModel.predict(dataSetWithoutTargetTest)
testAccuracy = accuracy_score(rfModelTestPred, targetTest)

print("Testing Data Accuracy: ", testAccuracy)
print(classification_report(targetTest, rfModelTestPred))

Testing Data Accuracy:  0.7704918032786885
              precision    recall  f1-score   support

           0       0.85      0.61      0.71        28
           1       0.73      0.91      0.81        33

    accuracy                           0.77        61
   macro avg       0.79      0.76      0.76        61
weighted avg       0.79      0.77      0.76        61



Simple Predictor Program for RF Model 

In [93]:
inputtedData = (58,1,1,125,220,0,1,144,0,0.4,1,4,3) #should mostly print out Person has heart disease
#inputtedData = (64,1,2,140,335,0,1,158,0,0,2,0,2) #should mostly print out Person does not have heart disease


reshapedNumPyArr = np.asarray(inputtedData).reshape(1,-1)

prediction = rfModel.predict(reshapedNumPyArr)

print(prediction)

if(prediction[0] == 0):
    print("Person does not have heart disease")
else:
    print("Person has heart disease")



[1]
Person has heart disease


# Evaluating Performance of All Models Based on New Data

In [94]:
import csv
import numpy as np
import warnings

warnings.filterwarnings("ignore")
with open('./hdp.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    output = list(reader)
output = output[1:]
data = [list(map(float, lst[:-1])) + [lst[-1]] for lst in output]

truelr = 0
trueknn = 0
truerf = 0

increment = 1

lrincorrect = []
knnincorrect = []
rfincorrect = []

for item in data:
    inputtedData = np.asarray(item[:-1]).reshape(1, -1)
    try:
        target = int(item[-1])
    except ValueError:
        if item[-1] == "Absence":
            target = 1
        else:
            target = 0

    lrprediction = LRmodel.predict(inputtedData)
    knnprediction = KNNModel.predict(inputtedData)
    rfprediction = rfModel.predict(inputtedData)

    if lrprediction[0] == target:
        truelr += 1
    else:
        lrincorrect.append("Line: " + str(increment) + " Target: " + str(target))

    if knnprediction[0] == target:
        trueknn += 1
    else:
        knnincorrect.append("Line: " + str(increment) + " Target: " + str(target))

    if rfprediction[0] == target:
        truerf += 1
    else:
        rfincorrect.append("Line: " + str(increment) + " Target: " + str(target))

    increment += 1

print(f"Correct Predictions (LR): {truelr}/{len(data)}")
print(f"Correct Predictions (KNN): {trueknn}/{len(data)}")
print(f"Correct Predictions (RF): {truerf}/{len(data)}")
print("Incorrect Predictions (LR):", lrincorrect)
print("Incorrect Predictions (KNN):", knnincorrect)
print("Incorrect Predictions (RF):", rfincorrect)


Correct Predictions (LR): 207/270
Correct Predictions (KNN): 191/270
Correct Predictions (RF): 216/270
Incorrect Predictions (LR): ['Line: 2 Target: 1', 'Line: 4 Target: 1', 'Line: 6 Target: 1', 'Line: 11 Target: 1', 'Line: 12 Target: 1', 'Line: 19 Target: 1', 'Line: 20 Target: 1', 'Line: 32 Target: 1', 'Line: 44 Target: 1', 'Line: 47 Target: 0', 'Line: 48 Target: 0', 'Line: 57 Target: 0', 'Line: 59 Target: 0', 'Line: 61 Target: 1', 'Line: 65 Target: 1', 'Line: 68 Target: 0', 'Line: 70 Target: 0', 'Line: 85 Target: 1', 'Line: 86 Target: 1', 'Line: 88 Target: 1', 'Line: 92 Target: 0', 'Line: 94 Target: 0', 'Line: 96 Target: 0', 'Line: 99 Target: 1', 'Line: 102 Target: 0', 'Line: 106 Target: 0', 'Line: 113 Target: 0', 'Line: 119 Target: 1', 'Line: 131 Target: 0', 'Line: 132 Target: 0', 'Line: 134 Target: 0', 'Line: 135 Target: 1', 'Line: 138 Target: 0', 'Line: 140 Target: 1', 'Line: 144 Target: 1', 'Line: 147 Target: 0', 'Line: 159 Target: 1', 'Line: 162 Target: 0', 'Line: 170 Target: 0'